In [1]:
# pip install PyPDF2
# pip install fpdf

######## ALTERNATIVE with annoying watermark
# pip install aspose-words

In [1]:
import os
import glob
import json
import ast
# import md2pdf

from openai import OpenAI

def load_api_key(config_file):
    with open(config_file) as f:
        config = json.load(f)
    client = OpenAI(api_key=config['api_key'])
    return client

def get_text_files(directory):
    list_of_files = glob.glob(os.path.join(directory, '*.txt'))
    latest_file, file_name = [""," "]
    if len(list_of_files) > 0:
        latest_file = max(list_of_files, key=os.path.getctime)
        full_name = os.path.basename(latest_file)
        file_name = os.path.splitext(full_name)
    return latest_file, file_name[0]

def read_file(file_path):
    with open(file_path, 'r') as f:
        content = f.read()
    return content

def output_directories(path,name,extension):
    if not os.path.exists(path):
        os.makedirs(path)
    n_files = ""
    # n_files = len(get_text_files(path))
    # n_files = n_files + 1
    # if n_files < 10:
    #     n_files = "0" + str(n_files)
    # else:
    #     n_files = str(n_files)
    nameFile = name + n_files + extension
    return(path + nameFile)

# from __future__ import print_function

def find_values(id, json_repr):
    results = []

    def _decode_dict(a_dict):
        try:
            results.append(a_dict[id])
        except KeyError:
            pass
        return a_dict

    json.loads(json_repr, object_hook=_decode_dict) # Return value ignored.
    return results

directory = "./mealPlans/"
config_file = "./noSubmit/config.json"

client = load_api_key(config_file)


In [2]:

def get_completion(prompt, client, model="gpt-4-1106-preview", temperature=0, ):

    messages = [{"role": "system", "content": 'You are a useful and concise inventory manager in a grocery store'},
                {"role": "user", "content": prompt}]
    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    # response = completion.model_dump_json(indent=2)
    return completion.choices[0].message.content # message["content"]



def get_completion_from_messages(messages, client, model="gpt-4", temperature=0):

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )

    return response.choices[0].message["content"]

In [3]:
###################################
####### Ingredients  ############
###################################
# INPUT: Lists of Ingredients for each day in Danish.
# OUTPUT: English translation of the lists of Ingredients for each day.

# Setup new file of MealPlans

text_file, file_name = get_text_files(directory)

output_Ingredients_dir = "./outputIngredients/"
outputIngredientsFile = output_directories(output_Ingredients_dir, file_name + "_Ingredients_",".txt")

content = read_file(text_file)
prompt = f"""
Your task is to translate the  
lists of ingredients from Danish to English.\

Lists of ingredients: 
"""

content_prompt = f"""
```{content}```
"""
response = get_completion(prompt+content_prompt, client)

print(response)

#######################################
#######################################
# Saving results

f = open(outputIngredientsFile, "w")
f.write(response)
f.close()  
    

Here are the translations of the ingredients from Danish to English:

**Indian Dahl**

INGREDIENTS

- 300 g carrots, grated
- 2 large onions
- 3 cloves of garlic
- 2 apples
- 0.5 tsp ground chili
- 1 tbsp cumin
- 1 tsp coriander
- 1 can chopped tomatoes
- 400 ml light coconut milk
- 4 dl red lentils, dried
- 2 dl water
- salt and pepper
- optionally 1 tsp ground or grated ginger if you have it
- 1 tbsp oil for frying
- 1 pack Naan bread
- cheasy fraiche 5%

**Red Rice Dish with Chicken and Bell Pepper**

INGREDIENTS

- 400 g chicken breast, tenderloin, or thigh fillet
- 2 onions
- 3 bell peppers
- 3 dl rice

**Bean "Meat"balls in Tomato Sauce**

INGREDIENTS

- 2 cans black beans
- 2 eggs
- 1 regular onion
- 100 g grated cheese
- 2 tbsp BBQ sauce
- 6 tbsp oatmeal
- 500 ml tomato passata
- 1 tbsp tomato paste
- 2 cloves of garlic
- 1 tbsp oregano
- 1 tsp thyme
- 300 g whole grain spaghetti or pasta

**Rye Bread with Homemade Tuna Salad**

INGREDIENTS

- 2 cans of tuna in water
- 2 tbsp f

In [4]:
###################################
####### ShoppingList  ############
###################################
# INPUT: Lists of Ingredients for each day in English.
# OUTPUT: Json object containing a table with the ingredients 
#   grouped by category and the quantities added up.

# Setup new file of ShoppingLists

text_file, file_name = get_text_files(output_Ingredients_dir)

output_ShoppingList_dir = "./outputShoppingList/"
outputShoppingListFile_txt = output_directories(output_ShoppingList_dir, file_name + "_ShoppingList_text",".txt")

content = read_file(text_file)
prompt = f"""

From the list of ingredients between <>, perform the following tasks: \
1. isolate the ingredients. \
2. group the ingredients by type and add up the quantities. \
3. make a shopping list grouping the type of ingredient using the following categories: \
- produce. \
- meat. \
- canned goods. \
- eggs, milk, yogurt, sour cream. \
- cheese. \
- cold cuts. \
- nuts. \
- spices. \
- 

If there are no ingredients in one category, do not list the category in the shopping list.

Output only the shopping list.

Lists of ingredients: 
"""

content_prompt = f"""
```{content}```
"""
response = get_completion(prompt+content_prompt, client)

print(response)

#######################################
#######################################
# Saving results

f = open(outputShoppingListFile_txt, "w")
f.write(response)
f.close()  
    

Produce:
- Carrots: 300 g + 4 = 304 g
- Onions: 2 large + 2 + 1 regular + 1 regular + 1 large + 2 regular = 9 onions
- Garlic: 3 cloves + 2 cloves + 3 cloves + 2 cloves = 10 cloves
- Apples: 2
- Bell peppers: 3 + 0.5 + 0.5 + 1 = 5
- Red lentils: 4 dl + 200 g
- Broccoli: 1 + 1 large
- Spring onions: 1 bunch
- Zucchini: 1
- Potatoes: 500 g
- Cucumber: 0.5

Meat:
- Chicken breast/tenderloin/thigh fillet: 400 g + 400 g = 800 g

Canned goods:
- Chopped tomatoes: 1 can + 2 cans
- Black beans: 2 cans
- Tuna in water: 2 cans
- Tomato passata: 500 ml

Eggs, milk, yogurt, sour cream:
- Eggs: 2
- Skim milk: 5 dl + 2.5 dl
- Fraiche 5%: 2 tbsp + 1 pack

Cheese:
- Grated cheese: 100 g + 75 g + 75 g

Nuts:
- (No items to list)

Spices:
- Ground chili: 0.5 tsp
- Cumin: 1 tbsp
- Coriander: 1 tsp
- Salt and pepper
- Ginger: 1 tsp (optional)
- Oregano: 1 tbsp
- Thyme: 1 tsp
- Bouillon cubes: 1 vegetable + 2
- Balsamic or HP sauce: 2 tbsp

Cold cuts:
- (No items to list)

Other:
- Red lentils: 4 dl + 200 

In [6]:
from fpdf import FPDF
# from PyPDF2 import PdfMerger 

def create_pdf(input_file,outputName):
    # Create a new FPDF object
    pdf = FPDF()

    # Open the text file and read its contents
    with open(input_file, 'r') as f:
        text = f.read()

    # Add a new page to the PDF
    pdf.add_page()

    # Set the font and font size
    pdf.set_font('Arial', size=12)

    # Write the text to the PDF
    pdf.write(5, text)

    # Save the PDF
    pdf.output(outputName)

    # # If a template PDF is specified, merge it with the new PDF
    # merger = PdfMerger()
    # template_pdf = './template.pdf'
    # if template_pdf:
    #     merger.append(PdfMerger(open(template_pdf, 'rb')))
    #     merger.append(PdfMerger(open('output.pdf', 'rb')))
    #     merger.write('merged_output.pdf')

In [7]:


create_pdf(outputShoppingListFile_txt, file_name+'shoppingList_2.pdf')

# UNTIL HERE

# STOP

# STOP

# STOP

In [8]:
# ###################################
# ####### ShoppingList  ############
# ###################################
# # INPUT: Lists of Ingredients for each day in English.
# # OUTPUT: Json object containing a table with the ingredients 
# #   grouped by category and the quantities added up.

# # Setup new file of ShoppingLists

# text_file, file_name = get_text_files(output_Ingredients_dir)

# output_ShoppingList_dir = "./outputShoppingList/"
# outputShoppingListFile_json = output_directories(output_ShoppingList_dir, file_name + "_ShoppingList_",".json")

# content = read_file(text_file)
# prompt = f"""
# Your task is to create a table with all the ingredients grouped
# by category and quantity.\

# The output table must be in Json format.\

# Your response must only be a Json object and nothing else.\

# Lists of ingredients: 
# """

# content_prompt = f"""
# ```{content}```
# """
# response = get_completion(prompt+content_prompt, client)

# print(response)

# #######################################
# #######################################
# # Saving results

# f = open(outputShoppingListFile_json, "w")
# f.write(response)
# f.close()  
    

In [9]:
# ###################################
# ####### ShoppingList  ############
# ###################################
# # INPUT: Lists of Ingredients for each day in English.
# # OUTPUT: Json object containing a table with the ingredients 
# #   grouped by category and the quantities added up.

# # Setup new file of ShoppingLists

# text_file, file_name = get_text_files(output_Ingredients_dir)

# output_ShoppingList_dir = "./outputShoppingList/"
# outputShoppingListFile_md = output_directories(output_ShoppingList_dir, file_name + "_ShoppingList_markdown",".md")

# content = read_file(text_file)
# prompt = f"""
# Create a shopping list with all the ingredients grouped
# by aisle and all the ingredients quantities added up together.\

# Lists of ingredients: 
# """

# content_prompt = f"""
# ```{content}```
# """
# response = get_completion(prompt+content_prompt, client)

# print(response)

# #######################################
# #######################################
# # Saving results

# f = open(outputShoppingListFile_md, "w")
# f.write(response)
# f.close()  
    

In [10]:
# # This code example demonstrates how to convert a Markdoen file to a PDF document.
# import aspose.words as aw

# # Load an existing Markdown document
# doc = aw.Document(outputShoppingListFile)

# # Save the document as PDF
# doc.save(file_name + "_ShoppingList_markdown.pdf")

In [11]:
# import json
# outputShoppingListFile = './outputShoppingList/w52_plan_Ingredients_03_ShoppingList_03.json'
# json_Shopping = json.loads(outputShoppingListFile)


In [12]:
# import pandas as pd
# import matplotlib.pyplot as plt

# df = pd.read_json(outputShoppingListFile)
# df.head()